In [89]:
# 학교알리미 공개용 데이터 중에서 서울시 중학교 졸업생의 진로현황 데이터셋을 사용하여 
# 고등학교 진학률이 비슷한 중학교끼리 군집(cluster)을 만들어 보자

import pandas as pd 
import folium 
# g학교알리미 공개용 데이터 중에서 서울시 중학교 졸업생의 진로현황 데이터셋
df = pd.read_excel('../data/2016_middle_shcool_graduates_report.xlsx', header=0)

In [90]:
# IPython console 디스플레이 옵션 설정하기
pd.set_option('display.width', None)                    # 출력화면의 너비
pd.set_option('display.max_rows', 100)                  # 출력할 행의 개수 한도
pd.set_option('display.max_columns', 30)                # 출력할 열의 개수 한도
pd.set_option('display.max_colwidth', 20)               # 출력할 열의 너비
pd.set_option('display.unicode.east_asian_width', True) # 유니코드 사용 너비 조정

# 데이터프레임의 열 이름 출력 
print(df.columns.values)

['Unnamed: 0' '지역' '학교명' '코드' '유형' '주야' '남학생수' '여학생수' '일반고' '특성화고' '과학고'
 '외고_국제고' '예고_체고' '마이스터고' '자사고' '자공고' '기타진학' '취업' '미상' '위도' '경도']


In [91]:
# 자료형 확인
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 21 columns):
Unnamed: 0    415 non-null int64
지역            415 non-null object
학교명           415 non-null object
코드            415 non-null int64
유형            415 non-null object
주야            415 non-null object
남학생수          415 non-null int64
여학생수          415 non-null int64
일반고           415 non-null float64
특성화고          415 non-null float64
과학고           415 non-null float64
외고_국제고        415 non-null float64
예고_체고         415 non-null float64
마이스터고         415 non-null float64
자사고           415 non-null float64
자공고           415 non-null float64
기타진학          415 non-null float64
취업            415 non-null int64
미상            415 non-null float64
위도            415 non-null float64
경도            415 non-null float64
dtypes: float64(12), int64(5), object(4)
memory usage: 68.2+ KB
None


In [92]:
# nan 데이터 포함 유무 확인
print(df.isnull().sum(axis=0))

Unnamed: 0     0
지역           0
학교명         0
코드           0
유형           0
주야           0
남학생수       0
여학생수       0
일반고         0
특성화고       0
과학고         0
외고_국제고    0
예고_체고      0
마이스터고     0
자사고         0
자공고         0
기타진학       0
취업           0
미상           0
위도           0
경도           0
dtype: int64


In [93]:
# 통계 요약정보 확인
print(df.describe())

Unnamed: 0        코드    남학생수    여학생수      일반고    특성화고  \
count  415.000000  415.000000  415.000000  415.000000  415.000000  415.000000   
mean   207.000000    3.197590  126.532530  116.173494    0.623080    0.149684   
std    119.944432    0.804272   79.217906   76.833082    0.211093    0.102977   
min      0.000000    3.000000    0.000000    0.000000    0.000000    0.000000   
25%    103.500000    3.000000   80.000000   71.500000    0.566500    0.065500   
50%    207.000000    3.000000  129.000000  118.000000    0.681000    0.149000   
75%    310.500000    3.000000  177.500000  161.500000    0.758000    0.224500   
max    414.000000    9.000000  337.000000  422.000000    0.908000    0.477000   

           과학고  외고_국제고   예고_체고  마이스터고      자사고  \
count  415.000000   415.000000  415.000000  415.000000  415.000000   
mean     0.004378     0.013687    0.017393    0.005251    0.080971   
std      0.006739     0.011548    0.092006    0.007557    0.079136   
min      0.000000     0.000000    

In [94]:
# 1. 지도에 위치 표시 
mschool_map = folium.Map(location=[37.55, 126.98], tiles='Stamen Terrain', zoom_start=12)

# 중학교 위치정보를 CirclMarker로 표시 
for name, lat, lng in zip(df.학교명, df.위도, df.경도):
    folium.CircleMarker([lat, lng],
        radius=5,               # 원의 반지름
        color='brown',          # 원의 둘레 색상
        fill=True,              
        fill_color='coral',     # 원을 채우는 색
        fill_opacity=0.7,       # 투명도
        popup=name              # 팝업기능(원형마커를 클릭하면 학교명이 팝업으로 출력)
        ).add_to(mschool_map)

mschool_map.save('seoul_mschool_location.html')

In [95]:
# 2. 데이터 전처리
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()    # label encdoer생성

# 모델이 인식할 수 없는 문자형 데이터를 원핫인코딩으로 처리하여 더미 변수에 저장
onehot_location = label_encoder.fit_transform(df['지역'])   # 지역구 이름
onehot_code = label_encoder.fit_transform(df['코드'])       # 3, 5, 9
onehot_type = label_encoder.fit_transform(df['유형'])       # 국립, 공립, 사립
onehot_day = label_encoder.fit_transform(df['주야'])        # 주간, 야간

# 원핫 인코딩된 결과를 새로운 열(변수)에 할당
df['location'] = onehot_location
df['code'] = onehot_code
df['type'] = onehot_type
df['day'] = onehot_day

In [96]:
print(df.head())

Unnamed: 0    지역                               학교명  코드  유형  주야  \
0           0  성북구  서울대학교사범대학부설중학교.....       3  국립  주간   
1           1  종로구  서울대학교사범대학부설여자중학교...     3  국립  주간   
2           2  강남구           개원중학교                     3  공립  주간   
3           3  강남구           개포중학교                     3  공립  주간   
4           4  서초구           경원중학교                     3  공립  주간   

   남학생수  여학생수  일반고  특성화고  과학고  외고_국제고  예고_체고  \
0       277         0   0.585     0.148   0.018        0.007      0.000   
1         0       256   0.680     0.199   0.000        0.035      0.008   
2       170       152   0.817     0.047   0.009        0.012      0.003   
3        83        72   0.755     0.097   0.013        0.013      0.019   
4       199       212   0.669     0.017   0.007        0.010      0.005   

   마이스터고  자사고  자공고  기타진학  취업   미상       위도        경도  \
0       0.011   0.227   0.000     0.004     0  0.000  37.594942  127.038909   
1       0.000   0.043   0.004     0.031     0  0.000  

In [97]:
# sklearn 라이브러리에서 cluster 군집 모델 가져오기 
from sklearn import cluster

# 분석1. 과학고, 외고_국제고, 자사고 진학률로 군집

# 분석에 사용할 속성을 선택(과학고, 외고_국제고, 자사고 진학률)
print('분석1.과학고, 외고_국제고, 자사고 진학률로 군집')
columns_list = [10, 11, 14]     # 각컬럼의 인덱스 번호(위치)
x = df.iloc[ : , columns_list]  # 행전체, 10, 11, 14 위치의 컬럼(열)
print(x[ :10])

분석1.과학고, 외고_국제고, 자사고 진학률로 군집
   과학고  외고_국제고  자사고
0   0.018        0.007   0.227
1   0.000        0.035   0.043
2   0.009        0.012   0.090
3   0.013        0.013   0.065
4   0.007        0.010   0.282
5   0.007        0.007   0.108
6   0.015        0.036   0.330
7   0.032        0.005   0.206
8   0.013        0.029   0.167
9   0.006        0.028   0.114


In [98]:
# 설명 변수 데이터를 정규화
x = preprocessing.StandardScaler().fit(x).transform(x)

# DBSCAN 모델 객체 생성
# 밀도 계산의 기준이 되는 반지름 R(eps=0.2)과 최소 포인트 개수 M(min_samples=5) 설정
# eps= 값에 따라 민감하게 반응. 
# 각 조건 별로 진행을 해봐야됨.
dbm = cluster.DBSCAN(eps=0.2, min_samples=5)

# DBSCAN 모델학습
dbm.fit(x)

# 예측 (굮집) 결과를 출력한 열(속성)의 값 구하기
# 모델의 labels_ 속성으로 확인하면 5개의 클러스터 값 ( -1, 0, 1, 2, 3 ) 으로 나타남
cluster_label = dbm.labels_
print(cluster_label) 
# -1, 0, 1, 2, 3 의 5가지의 클러스터

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  2 -1  0 -1
 -1 -1 -1 -1  0 -1 -1 -1 -1 -1  0  3 -1 -1 -1 -1 -1 -1 -1  0 -1 -1  1  0
 -1 -1 -1  0 -1 -1 -1 -1  0 -1  0  0 -1 -1  0 -1 -1 -1  0  0 -1 -1  0 -1
 -1 -1  0 -1 -1 -1  0  2  0  0  0  0  0 -1 -1 -1  0 -1  0 -1 -1  0 -1  0
 -1  0  0 -1 -1 -1 -1  1  0 -1  0  0 -1 -1 -1  0 -1 -1 -1 -1 -1  0  1 -1
 -1  0  2  0 -1 -1  1 -1 -1 -1  0  0  0 -1 -1  0 -1 -1 -1  0  0 -1 -1 -1
 -1  0 -1 -1 -1  0 -1 -1 -1  0 -1  0  0 -1 -1 -1 -1 -1  0 -1  0  0 -1 -1
 -1 -1 -1  0 -1 -1 -1  1  0  3  1 -1  0  0 -1  0 -1 -1  0  0  2 -1 -1  3
  0  0 -1 -1 -1 -1  0 -1  0  0 -1  0  0  0 -1 -1  0 -1 -1 -1 -1 -1  2  0
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1  0 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1  0  0 -1 -1  0 -1  3  0  2 -1 -1
 -1 -1  0 -1 -1 -1  0 -1  0  0 -1 -1 -1 -1 -1  1 -1  0  1 -1  0  0  1 -1
  2 -1  0 -1 -1 -1 -1  0 -1 -1  1  0 -1  0 -1 -1  0  3  0 -1 -1 -1  2 -1
 -1 -1 -1  0  0  0  1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1

In [99]:
# 예측(군집) 결과를 저장할 열(Cluster)을 데이터프레임에 추가 
df['Cluster'] = cluster_label       # cluster 열이 추가됨 
print(df.head())

Unnamed: 0    지역                               학교명  코드  유형  주야  \
0           0  성북구  서울대학교사범대학부설중학교.....       3  국립  주간   
1           1  종로구  서울대학교사범대학부설여자중학교...     3  국립  주간   
2           2  강남구           개원중학교                     3  공립  주간   
3           3  강남구           개포중학교                     3  공립  주간   
4           4  서초구           경원중학교                     3  공립  주간   

   남학생수  여학생수  일반고  특성화고  과학고  외고_국제고  예고_체고  \
0       277         0   0.585     0.148   0.018        0.007      0.000   
1         0       256   0.680     0.199   0.000        0.035      0.008   
2       170       152   0.817     0.047   0.009        0.012      0.003   
3        83        72   0.755     0.097   0.013        0.013      0.019   
4       199       212   0.669     0.017   0.007        0.010      0.005   

   마이스터고  자사고  자공고  기타진학  취업   미상       위도        경도  \
0       0.011   0.227   0.000     0.004     0  0.000  37.594942  127.038909   
1       0.000   0.043   0.004     0.031     0  0.000  

In [100]:
# 클러스터 값으로 그룹화를 하고, 그룹별로 내용 출력
grouped_cols = [1, 2, 4] + columns_list     # 인덱스(지역명, 학교명, 유형) + 기존 3개(10, 11, 14)
grouped = df.groupby('Cluster')         # 클러스터값 기준으로 그룹화시킴 
for key, group in grouped : 
    print('* key :', key)               # 클러스터 값: -1, 0, 1, 2, 3
    print('* number :', len(group))     # 각 클러스터 속한 학교수
    print(group.iloc[ : , grouped_cols].head())  # 5개의 데이터 출력
    print('\n')

# 클러스터 -1 : Outlier , 제외
# 클러스터 0 : 외고_국제고와 자사고 합격률은 높지만 과학고 합격자가 없다.
# 클러스터 1 : 자사고 합격자만 존재하는 그룹
# 클러스터 2 : 자사고 합격률이 매우 높으면서 과학고와 외고_국제고 합격자도 일부 존재한다.
# 클러스터 3 : 과학고 합격자 없이 외고_국제고와 자사고 합격자를 배출한 점은 클러스터 0과 비슷하지만, 외고_국제고 합격률이 클러스터 0에 비해 현저하게 낮다. 

* key : -1
* number : 255
     지역                               학교명  유형  과학고  외고_국제고  \
0  성북구  서울대학교사범대학부설중학교.....    국립   0.018        0.007   
1  종로구  서울대학교사범대학부설여자중학교...  국립   0.000        0.035   
2  강남구           개원중학교                  공립   0.009        0.012   
3  강남구           개포중학교                  공립   0.013        0.013   
4  서초구           경원중학교                  공립   0.007        0.010   

   자사고  
0   0.227  
1   0.043  
2   0.090  
3   0.065  
4   0.282  


* key : 0
* number : 102
      지역          학교명  유형  과학고  외고_국제고  자사고
13  서초구  동덕여자중학교  사립     0.0        0.022   0.038
22  강남구      수서중학교  공립     0.0        0.019   0.044
28  서초구      언남중학교  공립     0.0        0.015   0.050
34  강남구      은성중학교  사립     0.0        0.016   0.065
43  송파구      거원중학교  공립     0.0        0.021   0.054


* key : 1
* number : 45
         지역          학교명  유형  과학고  외고_국제고  자사고
46     강동구      동신중학교  사립     0.0          0.0   0.044
103    양천구      신원중학교  공립     0.0          0.0   0.006
118    구로구     

In [101]:
# 그래프로 표현 - 시각화 (클러스터별로)
colors = {-1:'gray', 0:'coral', 1:'blue', 2:'green', 3:'red', 4:'purple', 5:'orange', 6:'brown', 7:'brick', 8:'yellow', 9:'magenta', 10:'cyan'}

cluster_map = folium.Map(location=[37.55, 126.98], tiles='Stamen Terrain', zoom_start=12)

for name, lat, lng, clus in zip(df.학교명, df.위도, df.경도, df.Cluster):
    folium.CircleMarker([lat, lng],
    radius=5,                   # 원의 반지름
    color=colors[clus],         # 원의 둘레 색상
    fill=True,
    fill_color=colors[clus],    # 원을 채우는 색
    fill_opacity=0.7,           # 투명도
    popup=name
    ).add_to(cluster_map)


# 지도를 html 파일로 저장하기
cluster_map.save('seoul_mschool_cluster.html')

In [102]:
# 분석2. 과학고, 외고_국제고, 자사고 진학률, 유형(국립,공립,사립)으로 군집

# X2 데이터셋에 대하여 위의 과정을 반복(과학고, 외고_국제고, 자사고 진학률, 유형)
print('분석2. 과학고, 외고_국제고, 자사고 진학률, 유형(국립,공립,사립)으로 군집')
columns_list2 = [10, 11, 14, 23]
x2 = df.iloc[:, columns_list2]
print(x2[:5])

분석2. 과학고, 외고_국제고, 자사고 진학률, 유형(국립,공립,사립)으로 군집
   과학고  외고_국제고  자사고  type
0   0.018        0.007   0.227     1
1   0.000        0.035   0.043     1
2   0.009        0.012   0.090     0
3   0.013        0.013   0.065     0
4   0.007        0.010   0.282     0


In [103]:
# 설명 변수 데이터를 정규화
x2 = preprocessing.StandardScaler().fit(x2).transform(x2)

# DBSCAN 모델 객체 생성
# 밀도 계산의 기준이 되는 반지름 R(eps=0.2)과 최소 포인트 개수 M(min_samples=5) 설정
dbm2 = cluster.DBSCAN(eps=0.2, min_samples=5)

# DBSCAN 모델 학습
dbm2.fit(x2)

DBSCAN(algorithm='auto', eps=0.2, leaf_size=30, metric='euclidean',
       metric_params=None, min_samples=5, n_jobs=None, p=None)

In [104]:
# 예측(군집) 결과를 저장핛 열(Cluster2)을 데이터프레임에 추가
df['Cluster2'] = dbm2.labels_ # Cluster2 열 추가됨

# 클러스터 값으로 그룹화하고, 그룹별로 내용 출력 (첫 5행만 출력)
grouped2_cols = [1, 2, 4] + columns_list2   # 1:지역명, 2:학교명, 4:유형
grouped2 = df.groupby('Cluster2')

for key, group in grouped2:
    print('* key :', key)               # 클러스터 값: -1, 0 ~ 10
    print('* number :', len(group))     # 각 클러스터 속한 학교수
    print(group.iloc[:, grouped2_cols].head()) # 5개의 데이터 출력
    print('\n')

* key : -1
* number : 281
     지역                               학교명  유형  과학고  외고_국제고  \
0  성북구  서울대학교사범대학부설중학교.....    국립   0.018        0.007   
1  종로구  서울대학교사범대학부설여자중학교...  국립   0.000        0.035   
2  강남구           개원중학교                  공립   0.009        0.012   
3  강남구           개포중학교                  공립   0.013        0.013   
4  서초구           경원중학교                  공립   0.007        0.010   

   자사고  type  
0   0.227     1  
1   0.043     1  
2   0.090     0  
3   0.065     0  
4   0.282     0  


* key : 0
* number : 8
       지역      학교명  유형  과학고  외고_국제고  자사고  type
22   강남구  수서중학교  공립     0.0        0.019   0.044     0
43   송파구  거원중학교  공립     0.0        0.021   0.054     0
51   송파구  방이중학교  공립     0.0        0.021   0.068     0
93   강서구  방원중학교  공립     0.0        0.019   0.057     0
164  중랑구  원묵중학교  공립     0.0        0.020   0.062     0


* key : 1
* number : 59
      지역      학교명  유형  과학고  외고_국제고  자사고  type
28  서초구  언남중학교  공립     0.0        0.015   0.050     0
47  강동구  둔촌중학교  공립

In [105]:
cluster2_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', zoom_start=12)
for name, lat, lng, clus in zip(df.학교명, df.위도, df.경도, df.Cluster2):
    folium.CircleMarker([lat, lng],
    radius=5,       # 원의 반지름
    color=colors[clus], # 원의 둘레 색상
    fill=True,
    fill_color=colors[clus], # 원을 채우는 색
    fill_opacity=0.7, # 투명도
    popup=name
    ).add_to(cluster2_map)

# 지도를 html 파일로 저장하기
cluster2_map.save('seoul_mschool_cluster2.html')

In [107]:
# 분석3. 과학고, 외고_국제고 군집

# X3 데이터셋에 대하여 위의 과정을 반복(과학고, 외고_국제고)
print('분석3. 과학고, 외고_국제고 군집')
columns_list3 = [10, 11]
x3 = df.iloc[:, columns_list3]
print(x3[:5])

분석3. 과학고, 외고_국제고 군집
   과학고  외고_국제고
0   0.018        0.007
1   0.000        0.035
2   0.009        0.012
3   0.013        0.013
4   0.007        0.010


In [108]:
# 설명 변수 데이터를 정규화
x3 = preprocessing.StandardScaler().fit(x3).transform(x3)
# DBSCAN 모델 객체 생성
# 밀도 계산의 기준이 되는 반지름 R(eps=0.2)과 최소 포인트 개수 M(min_samples=5) 설정
dbm3 = cluster.DBSCAN(eps=0.2, min_samples=5)
# DBSCAN 모델 학습
dbm3.fit(x3)

# 예측(군집) 결과를 저장핛 열(Cluster3)을 데이터프레임에 추가
df['Cluster3'] = dbm3.labels_ # Cluster3 열 추가됨

# 클러스터 값으로 그룹화하고, 그룹별로 내용 출력 (첫 5행만 출력)
grouped3_cols = [1, 2, 4] + columns_list3 # 1:지역명, 2:학교명, 4:유형
grouped3 = df.groupby('Cluster3')
for key, group in grouped3:
    print('* key :', key) # 클러스터 값: -1, 0 ~ 6
    print('* number :', len(group)) # 각 클러스터 속핚 학교수
    print(group.iloc[:, grouped3_cols].head()) # 5개의 데이터 출력
    print('\n')

* key : -1
* number : 61
     지역                             학교명  유형  과학고  외고_국제고
0  성북구  서울대학교사범대학부설중학교.....  국립   0.018        0.007
3  강남구           개포중학교                공립   0.013        0.013
6  강남구         압구정중학교                공립   0.015        0.036
7  강남구  단국대학교사범대학부속중학교.....  사립   0.032        0.005
8  강남구           대명중학교                공립   0.013        0.029


* key : 0
* number : 160
      지역                               학교명  유형  과학고  외고_국제고
1   종로구  서울대학교사범대학부설여자중학교...  국립     0.0        0.035
13  서초구       동덕여자중학교                  사립     0.0        0.022
22  강남구           수서중학교                  공립     0.0        0.019
28  서초구           언남중학교                  공립     0.0        0.015
29  강남구           언북중학교                  공립     0.0        0.007


* key : 1
* number : 111
      지역      학교명  유형  과학고  외고_국제고
2   강남구  개원중학교  공립   0.009        0.012
4   서초구  경원중학교  공립   0.007        0.010
5   강남구  구룡중학교  공립   0.007        0.007
11  강남구  대치중학교  공립   0.007        0.024
14  서초

In [109]:
cluster3_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', zoom_start=12)
for name, lat, lng, clus in zip(df.학교명, df.위도, df.경도, df.Cluster3):
    folium.CircleMarker([lat, lng],
    radius=5,               # 원의 반지름
    color=colors[clus],     # 원의 둘레 색상
    fill=True,
    fill_color=colors[clus], # 원을 채우는 색
    fill_opacity=0.7,        # 투명도
    popup=name
    ).add_to(cluster3_map)

# 지도를 html 파일로 저장하기
cluster3_map.save('seoul_mschool_cluster3.html')